## Generate HScript function completions

In [1]:
import re
import json
import random
import subprocess
from pprint import pprint

In [2]:
# Obtain available functions with hbatch.
proc = subprocess.run('hbatch -q -c exhelp;quit'.split(),
                      stdout=subprocess.PIPE,
                      universal_newlines=True)
funcs = set(proc.stdout.split())

# Collect exhelp helpcards.
script = ';'.join(f'exhelp {f}' for f in sorted(funcs))
script += ';quit'
proc = subprocess.run(f'hbatch -q -c "{script}"',
                     stdout=subprocess.PIPE,
                     universal_newlines=True)
output = proc.stdout

In [3]:
data = re.sub(r'[`"<>]', '', output).strip().split('----\n')
print(f'Obtained {len(data)} expression functions')

Obtained 437 expression functions


In [4]:
functions = {}
ill = []
for helpcard in data:
    match = re.match(r'(\w+)\s*(\w+)\s*\((.*)?\)(?:.|\n)*?USAGE\s+\2\(((?:.|\n)*?)?\)', helpcard)
    if not match:
        ill.append(helpcard)
        continue
    extype = match[1]
    exname = match[2]

    argtypes = [a for a in (a.strip() for a in match[3].split(',')) if a]
    argnames = [a for a in (a.strip() for a in match[4].split(',')) if a]
    functions[exname] = extype, argtypes, argnames
    
    if exname == 'clamptosphere':
        print(argnames)

print(f'Parsed {len(functions)} functions, {len(ill)} helpcards cannot be parsed:')
for i in ill:
    first_line = i.split('\n', 1)[0]
    print(f"    {first_line}")

print('Overriding with:')
with open('overrides.cmd') as f:
    for over in f.read().split('\n'):
        print(f'    {over}')
        match = re.match(r'(\w+) (\w+)\((.*)\)', over)
        extype, exname, args = match.group(1, 2, 3)
        argtypes, argnames = [], []
        if args:
            argtypes, argnames = zip(*[pair.split() for pair in args.split(',')])
        functions[exname] = extype, argtypes, argnames
        
for exname, (extype, argtypes, argnames) in functions.items():
    if len(argtypes) != len(argnames):
        print(exname, extype, argtypes, argnames)
        raise RuntimeError
        
functions = {k: v for k, v in sorted(functions.items())}

print(f'Parsed {len(functions)} of {len(data)} total.')

['x', 'y', 'z', 'min_radius', 'max_radius', 'constant_type']
Parsed 423 functions, 14 helpcards cannot be parsed:
    float arclenD (string, float, float, float, float)
    float ch (string)
    float chrampt (string, float, float, float)
    string copmetas (string, string)
    float explodematrixpr (matrix, vector, vector, string, string, string)
    float imgbounds (string, string, string, float)
    string opstreamname (string)
    float property (string, float)
    float propertyf (string, float, float)
    string propertys (string, string)
    string propertysop (string, string)
    string propertysraw (string, string)
    float propertyt (string, float, float)
    float strlen (string)
Overriding with:
    float arclenD(string surface_node, float prim_num, float ustart, float ustop, float divs)
    float ceil(float number)
    float ch(string channel)
    float chrampt(string ramp_path, float position, float component_index, float time)
    string copmetas(string compositing_nod

In [5]:
expressions = {}

for exname, (extype, argtypes, argnames) in functions.items():
    expressions[exname] = {'return': extype}
    expressions[exname]['argtypes'] = argtypes
    expressions[exname]['argnames'] = argnames

def test():
    print(len(expressions))
    f = random.choice(list(expressions.keys()))
    print(f)
    pprint(expressions[f])
    
test()

439
primuv
{'argnames': ['surface_node',
              'prim_num',
              'attrib_name',
              'attrib_index',
              'u',
              'v'],
 'argtypes': ['string', 'float', 'string', 'float', 'float', 'float'],
 'return': 'float'}


In [6]:
comps = {
    'scope': 'source.hscript -string',
    'completions': []
}
unique_triggers = set()

for exname, exdata in expressions.items():
    argnames = exdata['argnames']
    trigger = f"{exname}({', '.join(argnames)})"

    # Completion exists.
    if trigger in unique_triggers:
        continue

    # Completion not exists. Make a new one.
    unique_triggers.add(trigger)

    # Build contents.
    cargs = [f'${{{i}:{arg}}}' for i, arg in enumerate(argnames, 1)]
    contents = f"{exname}({', '.join(cargs)})"

    comps['completions'].append({'trigger': trigger,
                                 'contents': contents})


print('Generated %d completions.' % len(unique_triggers))


# Write completions into a functions.sublime-completions file.
with open('expressions.sublime-completions', 'w') as f:
    json.dump(comps, f, indent=4)

Generated 439 completions.
